In [1]:
!pip install torch_snippets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 117.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 M

In [2]:
import os
from torch_snippets import *
from torchvision import transforms
from sklearn.model_selection import train_test_split
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import imgaug.augmenters as iaa

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
tfms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # imagenet
])

In [5]:
def conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )

In [6]:
def up_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
        nn.ReLU(inplace=True)
    )

In [7]:
from torchvision.models import vgg16_bn
class UNet(nn.Module):
    def __init__(self, pretrained=True, out_channels=2):
        super().__init__()

        self.encoder = vgg16_bn(pretrained=pretrained).features
        self.block1 = nn.Sequential(*self.encoder[:6])
        self.block2 = nn.Sequential(*self.encoder[6:13])
        self.block3 = nn.Sequential(*self.encoder[13:20])
        self.block4 = nn.Sequential(*self.encoder[20:27])
        self.block5 = nn.Sequential(*self.encoder[27:34])

        self.bottleneck = nn.Sequential(*self.encoder[34:])
        self.conv_bottleneck = conv(512, 1024)

        self.up_conv6 = up_conv(1024, 512)
        self.conv6 = conv(512 + 512, 512)
        self.up_conv7 = up_conv(512, 256)
        self.conv7 = conv(256 + 512, 256)
        self.up_conv8 = up_conv(256, 128)
        self.conv8 = conv(128 + 256, 128)
        self.up_conv9 = up_conv(128, 64)
        self.conv9 = conv(64 + 128, 64)
        self.up_conv10 = up_conv(64, 32)
        self.conv10 = conv(32 + 64, 32)
        self.conv11 = nn.Conv2d(32, out_channels, kernel_size=1)
    def forward(self, x):
        block1 = self.block1(x)
        block2 = self.block2(block1)
        block3 = self.block3(block2)
        block4 = self.block4(block3)
        block5 = self.block5(block4)

        bottleneck = self.bottleneck(block5)
        x = self.conv_bottleneck(bottleneck)

        x = self.up_conv6(x)
        x = torch.cat([x, block5], dim=1)
        x = self.conv6(x)

        x = self.up_conv7(x)
        x = torch.cat([x, block4], dim=1)
        x = self.conv7(x)

        x = self.up_conv8(x)
        x = torch.cat([x, block3], dim=1)
        x = self.conv8(x)

        x = self.up_conv9(x)
        x = torch.cat([x, block2], dim=1)
        x = self.conv9(x)

        x = self.up_conv10(x)
        x = torch.cat([x, block1], dim=1)
        x = self.conv10(x)

        x = self.conv11(x)

        return x

In [8]:
ce = nn.CrossEntropyLoss()
def UnetLoss(preds, targets):
    ce_loss = ce(preds, targets)
    acc = (torch.max(preds, 1)[1] == targets).float().mean()
    return ce_loss, acc

In [9]:
def train_batch(model, data, optimizer, criterion):
    model.train()
    ims, ce_masks = data
    _masks = model(ims)
    optimizer.zero_grad()
    loss, acc = criterion(_masks, ce_masks)
    loss.backward()
    optimizer.step()
    return loss.item(), acc.item()

@torch.no_grad()
def validate_batch(model, data, criterion):
    model.eval()
    ims, masks = data
    _masks = model(ims)
    loss, acc = criterion(_masks, masks)
    return loss.item(), acc.item()

In [10]:
model = UNet().to(device)
criterion = UnetLoss
optimizer = optim.Adam(model.parameters(), lr=1e-3)
n_epochs = 20

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:09<00:00, 56.3MB/s]


In [11]:
!pip install torch_summary
from torchsummary import summary
summary(model, torch.zeros(1,3,576,768))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 576, 768]        --
├─Sequential: 1                          []                        --
|    └─Conv2d: 2-1                       [-1, 64, 576, 768]        1,792
├─Sequential: 1                          []                        --
|    └─Conv2d: 2-2                       [-1, 64, 576, 768]        (recursive)
├─Sequential: 1                          []                        --
|    └─BatchNorm2d: 2-3                  [-1, 64, 576, 768]        128
├─Sequential: 1                          []                        --
|    └─BatchNorm2d: 2-4                  [-1, 64, 576, 768]        (recursive)
├─Sequential: 1                          []                        --
|    └─ReLU: 2-5                         [-1, 64, 576, 768]        --
├─Sequential: 1                   

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 64, 576, 768]        --
├─Sequential: 1                          []                        --
|    └─Conv2d: 2-1                       [-1, 64, 576, 768]        1,792
├─Sequential: 1                          []                        --
|    └─Conv2d: 2-2                       [-1, 64, 576, 768]        (recursive)
├─Sequential: 1                          []                        --
|    └─BatchNorm2d: 2-3                  [-1, 64, 576, 768]        128
├─Sequential: 1                          []                        --
|    └─BatchNorm2d: 2-4                  [-1, 64, 576, 768]        (recursive)
├─Sequential: 1                          []                        --
|    └─ReLU: 2-5                         [-1, 64, 576, 768]        --
├─Sequential: 1                          []                        --
|    └─ReLU: 2-6                         [-1, 64, 576, 768]    

In [12]:
model.load_state_dict(torch.load('/content/drive/MyDrive/vgg19_dd/model_scripted.pt' ,map_location=torch.device('cpu')))

<All keys matched successfully>

In [33]:
class ss(Dataset):
    def __init__(self, split):
        self.items = stems(f'/content/drive/MyDrive/gdsc-nu-ml-hackathon-bts-case-competition/{split}/imgs')
        self.split = split
    def __len__(self):
        return len(self.items)
    def __getitem__(self, ix):
        image = read(f'/content/drive/MyDrive/gdsc-nu-ml-hackathon-bts-case-competition/{self.split}/imgs/{self.items[ix]}.jpg', 1)
        return image, self.items[ix]
    def choose(self): return self[randint(len(self))]
    def collate_fn(self, batch):
        ims, ix = list(zip(*batch))
        ims = torch.cat([tfms(im.copy()/255.)[None] for im in ims]).float().to(device)
        return ims, ix 
test = ss('test')
test_dl = DataLoader(test, batch_size=1, shuffle=True, collate_fn=test.collate_fn)

In [ ]:
items = test.items
items

In [36]:
%cd /content/drive/MyDrive/gdsc-nu-ml-hackathon-bts-case-competition/test
%mkdir mask1
%cd mask1

/content/drive/MyDrive/gdsc-nu-ml-hackathon-bts-case-competition/test
mkdir: cannot create directory ‘mask1’: File exists
/content/drive/MyDrive/gdsc-nu-ml-hackathon-bts-case-competition/test/mask1


In [20]:
%cd 

/root


In [ ]:
%cd /content

In [38]:
subplot_list = []
from torchvision.utils import save_image
# Iterate through the images in test_dl
for im, ix in test_dl:
    _mask = model(im)
    
    # Take the maximum value along the channel dimension
    _, _mask = torch.max(_mask, dim=1)
    
    # Create a subplot of the image and mas
    subplot = _mask.permute(1,2,0).detach().cpu()[:,:,0]
    save_image(subplot.float(), f"/content/drive/MyDrive/gdsc-nu-ml-hackathon-bts-case-competition/test/mask1{ix}.png")